Single File Processing

In [1]:
import scipy.io.wavfile
import numpy as np  
import scipy.signal as signal  
from agc.utils import fft2melmx  
from agc.stft import stft, istft 
from agc import tf_agc
import os

# read audiofile
sr, d = scipy.io.wavfile.read('./test.wav')
data_type = d.dtype
# convert from int16 to float (-1,1) range
convert_16_bit = float(2 ** 15)
d = d / (convert_16_bit + 1.0)

(y, D, E) = tf_agc(d, sr, t_scale=1.0, f_scale=8.0, causal_tracking=True, plot=False)

y = np.int16(y / np.max(np.abs(y)) * convert_16_bit)
scipy.io.wavfile.write(f'./test_agc.wav', sr, y)

/tmp/ipykernel_1315628/4202664862.py:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, d = scipy.io.wavfile.read('./test.wav')


Batch Processing for Multiple Files

In [ ]:
import os
import scipy.io.wavfile
import numpy as np
from agc import tf_agc
import tqdm
import multiprocessing
from functools import partial

def process_file(wav_file, source_dir, dest_dir):
    """Process a single WAV file with AGC"""
    source_path = os.path.join(source_dir, wav_file)
    dest_path = os.path.join(dest_dir, wav_file)
    
    try:
        # Read audio file
        sr, d = scipy.io.wavfile.read(source_path)
        
        # Convert from int16 to float (-1,1) range
        convert_16_bit = float(2 ** 15)
        d = d / (convert_16_bit + 1.0)
        
        # Apply AGC (without plotting for batch processing)
        (y, D, E) = tf_agc(d, sr, t_scale=1.0, f_scale=8.0, causal_tracking=True, plot=False)

        # Convert back to int16 to save
        y = np.int16(y / np.max(np.abs(y)) * convert_16_bit)
        scipy.io.wavfile.write(dest_path, sr, y)
        return None
        
    except Exception as e:
        return f"Error processing {wav_file}: {e}"

def main():

    source_dir = "/data/jinming/CHiME6_eval/CHiME6/audio/CH1/eval"
    dest_dir = "/data/jinming/CHiME6_eval/CHiME6/audio/CH1/eval/AGC_noncausal"

    # Create destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    else:
        print(f"Directory {dest_dir} already exists. Files may be overwritten.")

    # Get all wav files
    wav_files = [f for f in os.listdir(source_dir) if f.endswith('.wav')]
    print(f"Found {len(wav_files)} WAV files to process.")

    # Determine number of processes
    num_processes = 10
    print(f"Using {num_processes} processes for parallel processing")
    
    # Create a partial function with the fixed parameters
    process_func = partial(process_file, source_dir=source_dir, dest_dir=dest_dir)
    
    # Process files in parallel with a progress bar
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = list(tqdm.tqdm(
            pool.imap(process_func, wav_files),
            total=len(wav_files),
            desc="Processing files"
        ))
    
    # Print any errors
    errors = [r for r in results if r is not None]
    if errors:
        print(f"Encountered {len(errors)} errors:")
        for error in errors:
            print(error)
    
    print(f"Processing complete. Processed files saved to {dest_dir}")

if __name__ == "__main__":
    main()

Directory /data/jinming/CHiME6_eval/CHiME6/audio/CH1/eval/AGC_noncausal already exists. Files may be overwritten.
Found 19 WAV files to process.
Using 10 processes for parallel processing


Processing files: 100%|██████████| 19/19 [01:05<00:00,  3.43s/it]


Processing complete. Processed files saved to /data/jinming/CHiME6_eval/CHiME6/audio/CH1/eval/AGC_noncausal
